In [1]:
%load_ext autotime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as sk
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
from bayes_opt import BayesianOptimization

In [2]:
# Load data
train_data = np.loadtxt('train_data.csv', delimiter=',')
train_labels = np.loadtxt('train_labels.csv', delimiter=',')
test_data = np.loadtxt('test_data.csv', delimiter=',')
submission_accu = pd.read_csv('dummy_solution_accuracy.csv')
submission_log = pd.read_csv('dummy_solution_logloss.csv')
# Scaler, feature engineer
train_data = RobustScaler().fit_transform(train_data)
test_data = RobustScaler().fit_transform(test_data)
# 
skf=StratifiedKFold(n_splits=5, random_state=12345, shuffle=False)

time: 3.34 s


In [3]:
# Tuning xgboost with Bayes_opt - Accuracy - 'reg:logistic' # 30 iters
def xgbcv(learning_rate,n_estimators,gamma,max_depth,min_child_weight,max_delta_step,
          subsample,colsample_bytree,reg_lambda,reg_alpha):
    xgb_model =  xgb.XGBClassifier(
                         objective = 'reg:logistic',
                         learning_rate= max(min(learning_rate, 1), 0), 
                         n_estimators = int(n_estimators),
                         gamma=max(gamma, 0), 
                         max_depth = int(max_depth),
                         min_child_weight = int(min_child_weight),
                         max_delta_step= int(max_delta_step),
                         subsample = max(min(subsample, 1), 0),
                         colsample_bytree = max(min(colsample_bytree, 1), 0),
                         reg_lambda = max(reg_lambda, 0),
                         reg_alpha = max(reg_alpha, 0),
                         random_state=12345)
    val = cross_val_score(xgb_model,
            train_data, train_labels, scoring='accuracy',
            cv=skf).mean()
    return val
gp_params = {"alpha": 1e-5}
xgbBO = BayesianOptimization(xgbcv,
           {'learning_rate': (0.001, 1), 
            'n_estimators': (1, 100),
            'gamma': (0.001, 10),
            'max_depth': (1, 15),
            'min_child_weight': (1, 10),
            'max_delta_step': (1, 10),
            'subsample': (0.3, 1),
            'colsample_bytree': (0.001, 1),
            'reg_lambda': (0.001, 10),
            'reg_alpha': (0.001, 10)}
                            )
xgbBO.maximize(n_iter=30,acq="ei", xi=0.1, **gp_params)
print('-' * 53)
print('Final Results')
print('XGB: %f' % xgbBO.res['max']['max_val'])

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 02m55s |    0.58810 |             0.5639 |    8.3611 |          0.2552 |           3.4072 |     12.0649 |             2.3074 |        80.1424 |      6.1046 |       5.4135 |      0.3074 | 
    2 | 00m08s |    0.55600 |             0.6886 |    6.1697 |          0.8362 |           4.1607 |      4.7193 |             2.7207 |         3.3142 |      9.6426 |       3.6573 |      0.4821 | 
    3 | 00m03s |    0.54067 |             0.9867 |    4.9462 |          0.5720 |           8.7119 |     10.5864 |             6.7953 |         1.5243 |      9.7733 |       9.0265 |      0.3597 | 
    4 

In [4]:
# Tuning xgboost with randomsearch - Accuracy - 'reg:logistic'  # 30 iters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
xgb_model = xgb.XGBClassifier(objective = 'reg:logistic')
params={'learning_rate': uniform(0,1), 
        'n_estimators': randint(1, 100),
        'gamma': uniform(0.001, 10),
        'max_depth': randint(1, 15),
        'min_child_weight': randint(1, 10),
        'max_delta_step': randint(1, 10),
        'subsample': uniform(0.3, 0.7),
        'colsample_bytree': uniform(0, 1),
        'reg_lambda': uniform(1, 9),
        'reg_alpha': uniform(1, 9)
}
clf = RandomizedSearchCV(estimator= xgb_model, param_distributions=params, n_iter=30, 
                         cv= skf, scoring='accuracy',verbose = 3,  random_state = 12345)
clf.fit(train_data, train_labels)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Best scores set found on development set:")
print()
print(clf.best_score_)
print()
print("All scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.621436716077537, total=  26.0s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.1s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6327231121281465, total=  28.0s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   54.1s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6219931271477663, total=  26.7s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6206896551724138, total=  27.4s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.

...

[CV]  colsample_bytree=0.0651175403153, gamma=5.18198540569, learning_rate=0.59548562876, max_delta_step=3, max_depth=8, min_child_weight=2, n_estimators=92, reg_alpha=1.62576699539, reg_lambda=7.34926094737, subsample=0.628415697245, score=0.6052934407364787, total=  16.9s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955 
[CV]  colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955, score=0.5952109464082098, total=   9.4s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsampl

[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=0.5995423340961098, total=   4.3s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534 
[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=0.6013745704467354, total=   4.5s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsampl

[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=0.6068965517241379, total=  53.6s
[CV] colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804 
[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=0.5949367088607594, total=  52.2s
[CV] colsample_bytree=0.327113146299, gamma=2.55207179264, learning_rate=0.495002174357, max_delta_step=4, max_depth=4, min_child_weight=3, n_estimators=42, reg_alpha=8.17818603995, reg_lambda=4.43350470298, subsample=0

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 60.7min finished


Best parameters set found on development set:

{'colsample_bytree': 0.47861805353537923, 'gamma': 2.1254272871651136, 'learning_rate': 0.26547834158437322, 'max_delta_step': 3, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 52, 'reg_alpha': 1.6171576169146353, 'reg_lambda': 3.2107454434985847, 'subsample': 0.94424876686421366}

Best scores set found on development set:

0.643135457254

All scores on development set:

0.620 (+/-0.018) for {'colsample_bytree': 0.92961609281714785, 'gamma': 3.1647555458178589, 'learning_rate': 0.18391881167709445, 'max_delta_step': 6, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 60, 'reg_alpha': 9.3136434163165074, 'reg_lambda': 4.3652905813018137, 'subsample': 0.40848134922534496}

0.642 (+/-0.024) for {'colsample_bytree': 0.89234372944613349, 'gamma': 0.26889724889310418, 'learning_rate': 0.29150238712463061, 'max_delta_step': 3, 'max_depth': 7, 'min_child_weight': 8, 'n_estimators': 44, 'reg_alpha': 6.6438489428781935, 'reg_lambda': 9

In [5]:
# Tuning xgboost with Bayes_opt - Accuracy - 'multi:softmax' # 30 iters
def xgbcv(learning_rate,n_estimators,gamma,max_depth,min_child_weight,max_delta_step,
          subsample,colsample_bytree,reg_lambda,reg_alpha):
    xgb_model =  xgb.XGBClassifier(
                         objective = 'multi:softmax',
                         learning_rate= max(min(learning_rate, 1), 0), 
                         n_estimators = int(n_estimators),
                         gamma=max(gamma, 0), 
                         max_depth = int(max_depth),
                         min_child_weight = int(min_child_weight),
                         max_delta_step= int(max_delta_step),
                         subsample = max(min(subsample, 1), 0),
                         colsample_bytree = max(min(colsample_bytree, 1), 0),
                         reg_lambda = max(reg_lambda, 0),
                         reg_alpha = max(reg_alpha, 0),
                         random_state=12345)
    val = cross_val_score(xgb_model,
            train_data, train_labels, scoring='accuracy',
            cv=skf).mean()
    return val
gp_params = {"alpha": 1e-5}
xgbBO = BayesianOptimization(xgbcv,
           {'learning_rate': (0.001, 1), 
            'n_estimators': (1, 100),
            'gamma': (0.001, 10),
            'max_depth': (1, 15),
            'min_child_weight': (1, 10),
            'max_delta_step': (1, 10),
            'subsample': (0.3, 1),
            'colsample_bytree': (0.001, 1),
            'reg_lambda': (0.001, 10),
            'reg_alpha': (0.001, 10)}
                            )
xgbBO.maximize(n_iter=30,acq="ei", xi=0.1, **gp_params)
print('-' * 53)
print('Final Results')
print('XGB: %f' % xgbBO.res['max']['max_val'])

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 00m20s |    0.58879 |             0.9455 |    0.7338 |          0.0957 |           2.2603 |      4.2546 |             1.6664 |         7.8346 |      4.5402 |       4.2287 |      0.4043 | 
    2 | 00m24s |    0.58901 |             0.5710 |    6.2399 |          0.1834 |           3.6784 |      2.8541 |             6.1950 |        15.2304 |      0.4837 |       6.4594 |      0.7221 | 
    3 | 01m12s |    0.59658 |             0.2929 |    7.9137 |          0.6813 |           2.6653 |      3.6344 |             4.6190 |        65.3227 |      8.2205 |       9.2786 |      0.5004 | 
    4 

/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  3.89550009e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 45, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


   18 | 01m27s |    0.62091 |             0.6769 |    0.2281 |          0.8693 |           9.9338 |      4.7548 |             9.0312 |        36.9932 |      0.5081 |       0.2636 |      0.9137 | 
   19 | 00m39s |    0.59981 |             0.0531 |    5.8177 |          0.1870 |           9.5177 |     14.4686 |             9.6606 |        64.2858 |      0.6427 |       9.5476 |      0.3296 | 
   20 | 00m23s |    0.60186 |             0.1313 |    0.1692 |          0.4225 |           1.5446 |      1.8244 |             9.4781 |        40.1807 |      9.8722 |       9.7087 |      0.6135 | 
   21 | 00m57s |    0.60438 |             0.1886 |    0.0719 |          0.6486 |           5.8749 |     14.8399 |             1.6726 |        61.5115 |      9.5759 |       0.5097 |      0.3378 | 
   22 | 00m06s |    0.52211 |             0.2998 |    9.1258 |          0.3321 |           1.6265 |     13.7608 |             1.0290 |         1.3362 |      8.8561 |       9.5783 |      0.8124 | 
   23 | 00m39s |    

/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  1.47040937e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


   26 | 00m21s |    0.55648 |             0.0038 |    0.4983 |          0.5717 |           1.1117 |     14.6369 |             9.9641 |        41.7233 |      1.9927 |       2.2917 |      0.7919 | 
   27 | 01m51s |    0.62436 |             0.8127 |    0.0819 |          0.7582 |           9.9547 |      3.1598 |             1.3080 |        51.3297 |      0.3095 |       9.9327 |      0.4110 | 
   28 | 01m20s |    0.61216 |             0.9807 |    9.9271 |          0.4130 |           9.8576 |      1.5372 |             9.0357 |        57.1574 |      0.9868 |       1.3937 |      0.8412 | 
   29 | 11m39s |    0.64267 |             0.9294 |    0.7713 |          0.1105 |           1.1391 |     14.4515 |             3.0198 |        77.9721 |      0.0947 |       9.8953 |      0.9504 | 
   30 | 01m15s |    0.54321 |             0.6949 |    9.3270 |          0.8921 |           9.0086 |     14.6299 |             1.0478 |        17.3179 |      0.3502 |       1.0832 |      0.4593 | 
   31 | 00m44s |    

/opt/conda/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -2.36347437e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


   35 | 00m30s |    0.57618 |             0.4552 |    8.8895 |          0.1929 |           9.7070 |      1.0292 |             1.6434 |        30.6795 |      9.6828 |       9.5131 |      0.8149 | 
-----------------------------------------------------
Final Results
XGB: 0.651155
time: 1h 6min 48s


In [6]:
# Tuning xgboost with randomsearch - Accuracy - 'multi:softmax' # 30 iters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
xgb_model = xgb.XGBClassifier(objective = 'multi:softmax')
params={'learning_rate': uniform(0,1), 
        'n_estimators': randint(1, 100),
        'gamma': uniform(0.001, 10),
        'max_depth': randint(1, 15),
        'min_child_weight': randint(1, 10),
        'max_delta_step': randint(1, 10),
        'subsample': uniform(0.3, 0.7),
        'colsample_bytree': uniform(0, 1),
        'reg_lambda': uniform(1, 9),
        'reg_alpha': uniform(1, 9)
}
clf = RandomizedSearchCV(estimator= xgb_model, param_distributions=params, n_iter=30, 
                         cv= skf, scoring='accuracy', verbose = 3, random_state = 12345)
clf.fit(train_data, train_labels)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Best scores set found on development set:")
print()
print(clf.best_score_)
print()
print("All scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.621436716077537, total=  25.6s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   25.7s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6327231121281465, total=  27.1s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   52.9s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6219931271477663, total=  26.7s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=0.6206896551724138, total=  25.9s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.

...

[CV]  colsample_bytree=0.0651175403153, gamma=5.18198540569, learning_rate=0.59548562876, max_delta_step=3, max_depth=8, min_child_weight=2, n_estimators=92, reg_alpha=1.62576699539, reg_lambda=7.34926094737, subsample=0.628415697245, score=0.6052934407364787, total=  15.6s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955 
[CV]  colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955, score=0.5952109464082098, total=   8.2s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsampl

[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=0.5995423340961098, total=   4.2s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534 
[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=0.6013745704467354, total=   4.9s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsampl

[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=0.6068965517241379, total=  52.4s
[CV] colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804 
[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=0.5949367088607594, total=  52.3s
[CV] colsample_bytree=0.327113146299, gamma=2.55207179264, learning_rate=0.495002174357, max_delta_step=4, max_depth=4, min_child_weight=3, n_estimators=42, reg_alpha=8.17818603995, reg_lambda=4.43350470298, subsample=0

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 60.5min finished


Best parameters set found on development set:

{'colsample_bytree': 0.47861805353537923, 'gamma': 2.1254272871651136, 'learning_rate': 0.26547834158437322, 'max_delta_step': 3, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 52, 'reg_alpha': 1.6171576169146353, 'reg_lambda': 3.2107454434985847, 'subsample': 0.94424876686421366}

Best scores set found on development set:

0.643135457254

All scores on development set:

0.620 (+/-0.018) for {'colsample_bytree': 0.92961609281714785, 'gamma': 3.1647555458178589, 'learning_rate': 0.18391881167709445, 'max_delta_step': 6, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 60, 'reg_alpha': 9.3136434163165074, 'reg_lambda': 4.3652905813018137, 'subsample': 0.40848134922534496}

0.642 (+/-0.024) for {'colsample_bytree': 0.89234372944613349, 'gamma': 0.26889724889310418, 'learning_rate': 0.29150238712463061, 'max_delta_step': 3, 'max_depth': 7, 'min_child_weight': 8, 'n_estimators': 44, 'reg_alpha': 6.6438489428781935, 'reg_lambda': 9

In [ ]:
# Model 1 (accuracy)
xgbparams = {'num_class': 10}
xgb_model =  xgb.XGBClassifier(
                         objective = 'reg:logistic',
                         learning_rate= 0.1, 
                         n_estimators = 50,
                         gamma=0.1, 
                         max_depth = 12,
                         min_child_weight = 20,
                         max_delta_step= 50,
                         subsample = 0.9,
                         colsample_bytree = 0.7,
                         reg_lambda = 0.1,
                         reg_alpha = 0.1,
                         random_state=12345,
                         **xgbparams)
xgb_model = xgb_model.fit(train_data, train_labels, eval_metric = 'merror')
results = xgb_model.predict(test_data)
submission_accu.iloc[:,1] = results
submission_accu.to_csv('xgb_solution_accu_1.csv', index= False)

Score: 0.64654

In [7]:
# Tuning xgboost with Bayes_opt - logloss - 'reg:logistic' # 30 iters
def xgbcv(learning_rate,n_estimators,gamma,max_depth,min_child_weight,max_delta_step,
          subsample,colsample_bytree,reg_lambda,reg_alpha):
    xgb_model =  xgb.XGBClassifier(
                         objective = 'reg:logistic',
                         learning_rate= max(min(learning_rate, 1), 0), 
                         n_estimators = int(n_estimators),
                         gamma=max(gamma, 0), 
                         max_depth = int(max_depth),
                         min_child_weight = int(min_child_weight),
                         max_delta_step= int(max_delta_step),
                         subsample = max(min(subsample, 1), 0),
                         colsample_bytree = max(min(colsample_bytree, 1), 0),
                         reg_lambda = max(reg_lambda, 0),
                         reg_alpha = max(reg_alpha, 0),
                         random_state=12345)
    val = cross_val_score(xgb_model,
            train_data, train_labels, scoring='neg_log_loss',
            cv=skf).mean()
    return val
gp_params = {"alpha": 1e-5}
xgbBO = BayesianOptimization(xgbcv,
           {'learning_rate': (0.001, 1), 
            'n_estimators': (1, 100),
            'gamma': (0.001, 10),
            'max_depth': (1, 15),
            'min_child_weight': (1, 10),
            'max_delta_step': (1, 10),
            'subsample': (0.3, 1),
            'colsample_bytree': (0.001, 1),
            'reg_lambda': (0.001, 10),
            'reg_alpha': (0.001, 10)}
                            )
xgbBO.maximize(n_iter=30, acq="ei", xi=0.1,**gp_params)
print('-' * 53)
print('Final Results')
print('XGB: %f' % xgbBO.res['max']['max_val'])

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 02m17s |   -1.25952 |             0.7186 |    7.7209 |          0.3512 |           4.7566 |      2.7191 |             6.1214 |        69.9188 |      9.6869 |       5.3564 |      0.9751 | 
    2 | 00m47s |   -1.17293 |             0.6508 |    4.3360 |          0.9205 |           2.3378 |      2.7387 |             9.9008 |        29.8852 |      0.1061 |       8.8027 |      0.8754 | 
    3 | 00m24s |   -1.29224 |             0.7315 |    4.6164 |          0.8564 |           9.7859 |      2.6567 |             9.6871 |        21.6105 |      6.5887 |       3.9417 |      0.3420 | 
    4 

In [8]:
# Tuning xgboost with randomsearch - logloss - 'reg:logistic'  # 30 iters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
xgb_model = xgb.XGBClassifier(objective = 'reg:logistic')
params={'learning_rate': uniform(0,1), 
        'n_estimators': randint(1, 100),
        'gamma': uniform(0.001, 10),
        'max_depth': randint(1, 15),
        'min_child_weight': randint(1, 10),
        'max_delta_step': randint(1, 10),
        'subsample': uniform(0.3, 0.7),
        'colsample_bytree': uniform(0, 1),
        'reg_lambda': uniform(1, 9),
        'reg_alpha': uniform(1, 9)
}
clf = RandomizedSearchCV(estimator= xgb_model, param_distributions=params, n_iter=30, 
                         cv= skf, scoring='neg_log_loss',verbose = 3,  random_state = 12345)
clf.fit(train_data, train_labels)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Best scores set found on development set:")
print()
print(clf.best_score_)
print()
print("All scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2079292764778895, total=  29.6s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.7s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2126314070723627, total=  26.7s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   56.5s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2279063340613592, total=  27.8s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2075796530678355, total=  28.0s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=

...

[CV]  colsample_bytree=0.0651175403153, gamma=5.18198540569, learning_rate=0.59548562876, max_delta_step=3, max_depth=8, min_child_weight=2, n_estimators=92, reg_alpha=1.62576699539, reg_lambda=7.34926094737, subsample=0.628415697245, score=-1.1896292846403391, total=  17.5s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955 
[CV]  colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955, score=-1.3241682065121974, total=   9.7s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsam

[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=-1.2748503996575042, total=   4.0s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534 
[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=-1.3068572713183784, total=   4.2s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsam

[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=-1.2196626608527597, total=  53.9s
[CV] colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804 
[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=-1.2521989682970098, total=  53.8s
[CV] colsample_bytree=0.327113146299, gamma=2.55207179264, learning_rate=0.495002174357, max_delta_step=4, max_depth=4, min_child_weight=3, n_estimators=42, reg_alpha=8.17818603995, reg_lambda=4.43350470298, subsample

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 61.3min finished


Best parameters set found on development set:

{'colsample_bytree': 0.47861805353537923, 'gamma': 2.1254272871651136, 'learning_rate': 0.26547834158437322, 'max_delta_step': 3, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 52, 'reg_alpha': 1.6171576169146353, 'reg_lambda': 3.2107454434985847, 'subsample': 0.94424876686421366}

Best scores set found on development set:

-1.12075057595

All scores on development set:

-1.217 (+/-0.020) for {'colsample_bytree': 0.92961609281714785, 'gamma': 3.1647555458178589, 'learning_rate': 0.18391881167709445, 'max_delta_step': 6, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 60, 'reg_alpha': 9.3136434163165074, 'reg_lambda': 4.3652905813018137, 'subsample': 0.40848134922534496}

-1.126 (+/-0.031) for {'colsample_bytree': 0.89234372944613349, 'gamma': 0.26889724889310418, 'learning_rate': 0.29150238712463061, 'max_delta_step': 3, 'max_depth': 7, 'min_child_weight': 8, 'n_estimators': 44, 'reg_alpha': 6.6438489428781935, 'reg_lambda':

In [ ]:
# Tuning xgboost with Bayes_opt - logloss - 'multi:softprob' # 30 iters
def xgbcv(learning_rate,n_estimators,gamma,max_depth,min_child_weight,max_delta_step,
          subsample,colsample_bytree,reg_lambda,reg_alpha):
    xgb_model =  xgb.XGBClassifier(
                         objective = 'multi:softprob',
                         learning_rate= max(min(learning_rate, 1), 0), 
                         n_estimators = int(n_estimators),
                         gamma=max(gamma, 0), 
                         max_depth = int(max_depth),
                         min_child_weight = int(min_child_weight),
                         max_delta_step= int(max_delta_step),
                         subsample = max(min(subsample, 1), 0),
                         colsample_bytree = max(min(colsample_bytree, 1), 0),
                         reg_lambda = max(reg_lambda, 0),
                         reg_alpha = max(reg_alpha, 0),
                         random_state=12345)
    val = cross_val_score(xgb_model,
            train_data, train_labels, scoring='neg_log_loss',
            cv=skf).mean()
    return val
gp_params = {"alpha": 1e-5}
xgbBO = BayesianOptimization(xgbcv,
           {'learning_rate': (0.001, 1), 
            'n_estimators': (1, 100),
            'gamma': (0.001, 10),
            'max_depth': (1, 15),
            'min_child_weight': (1, 10),
            'max_delta_step': (1, 10),
            'subsample': (0.3, 1),
            'colsample_bytree': (0.001, 1),
            'reg_lambda': (0.001, 10),
            'reg_alpha': (0.001, 10)}
                            )
xgbBO.maximize(n_iter=30,acq="ei", xi=0.1, **gp_params)
print('-' * 53)
print('Final Results')
print('XGB: %f' % xgbBO.res['max']['max_val'])

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators |   reg_alpha |   reg_lambda |   subsample | 
    1 | 03m28s |   -1.32721 |             0.9752 |    6.7823 |          0.5752 |           8.5085 |      4.5222 |             3.2014 |        85.2757 |      9.6124 |       9.0011 |      0.3428 | 
    2 | 01m54s |   -1.13882 |             0.8219 |    1.2322 |          0.4132 |           5.5821 |     13.1158 |             8.4021 |        23.9894 |      1.2876 |       7.3750 |      0.8233 | 
    3 | 01m06s |   -1.24310 |             0.4738 |    3.8374 |          0.3789 |           1.1223 |     10.9509 |             6.7157 |        29.4168 |      9.9388 |       6.7961 |      0.5145 | 
    4 

In [3]:
# Tuning xgboost with randomsearch - logloss - 'multi:softprob'  # 30 iters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
xgb_model = xgb.XGBClassifier(objective = 'multi:softprob')
params={'learning_rate': uniform(0,1), 
        'n_estimators': randint(1, 100),
        'gamma': uniform(0.001, 10),
        'max_depth': randint(1, 15),
        'min_child_weight': randint(1, 10),
        'max_delta_step': randint(1, 10),
        'subsample': uniform(0.3, 0.7),
        'colsample_bytree': uniform(0, 1),
        'reg_lambda': uniform(1, 9),
        'reg_alpha': uniform(1, 9)
}
clf = RandomizedSearchCV(estimator= xgb_model, param_distributions=params, n_iter=30, 
                         cv= skf, scoring='neg_log_loss',verbose = 3,  random_state = 12345)
clf.fit(train_data, train_labels)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Best scores set found on development set:")
print()
print(clf.best_score_)
print()
print("All scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2079292764778895, total=  26.3s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.4s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2126314070723627, total=  26.6s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   53.1s remaining:    0.0s


[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2279063340613592, total=  26.3s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225 
[CV]  colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=0.408481349225, score=-1.2075796530678355, total=  26.3s
[CV] colsample_bytree=0.929616092817, gamma=3.16475554582, learning_rate=0.183918811677, max_delta_step=6, max_depth=3, min_child_weight=2, n_estimators=60, reg_alpha=9.31364341632, reg_lambda=4.3652905813, subsample=

...

[CV]  colsample_bytree=0.0651175403153, gamma=5.18198540569, learning_rate=0.59548562876, max_delta_step=3, max_depth=8, min_child_weight=2, n_estimators=92, reg_alpha=1.62576699539, reg_lambda=7.34926094737, subsample=0.628415697245, score=-1.1896292846403391, total=  15.8s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955 
[CV]  colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsample=0.474948842955, score=-1.3241682065121974, total=   8.7s
[CV] colsample_bytree=0.0101882161253, gamma=7.74923862866, learning_rate=0.79420100836, max_delta_step=2, max_depth=4, min_child_weight=6, n_estimators=90, reg_alpha=3.35956957989, reg_lambda=5.11182552498, subsam

[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=-1.2748503996575042, total=   4.4s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534 
[CV]  colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsample=0.389280383534, score=-1.3068572713183784, total=   4.3s
[CV] colsample_bytree=0.204904986418, gamma=4.65981544215, learning_rate=0.763071054182, max_delta_step=1, max_depth=1, min_child_weight=1, n_estimators=35, reg_alpha=7.94236333781, reg_lambda=3.29870559128, subsam

[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=-1.2196626608527597, total=  50.4s
[CV] colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804 
[CV]  colsample_bytree=0.532452009138, gamma=6.94815563668, learning_rate=0.243977061043, max_delta_step=9, max_depth=6, min_child_weight=5, n_estimators=77, reg_alpha=7.40908148615, reg_lambda=9.25703646324, subsample=0.86250975804, score=-1.2521989682970098, total=  50.7s
[CV] colsample_bytree=0.327113146299, gamma=2.55207179264, learning_rate=0.495002174357, max_delta_step=4, max_depth=4, min_child_weight=3, n_estimators=42, reg_alpha=8.17818603995, reg_lambda=4.43350470298, subsample

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 57.5min finished


Best parameters set found on development set:

{'colsample_bytree': 0.47861805353537923, 'gamma': 2.1254272871651136, 'learning_rate': 0.26547834158437322, 'max_delta_step': 3, 'max_depth': 8, 'min_child_weight': 4, 'n_estimators': 52, 'reg_alpha': 1.6171576169146353, 'reg_lambda': 3.2107454434985847, 'subsample': 0.94424876686421366}

Best scores set found on development set:

-1.12075057595

All scores on development set:

-1.217 (+/-0.020) for {'colsample_bytree': 0.92961609281714785, 'gamma': 3.1647555458178589, 'learning_rate': 0.18391881167709445, 'max_delta_step': 6, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 60, 'reg_alpha': 9.3136434163165074, 'reg_lambda': 4.3652905813018137, 'subsample': 0.40848134922534496}

-1.126 (+/-0.031) for {'colsample_bytree': 0.89234372944613349, 'gamma': 0.26889724889310418, 'learning_rate': 0.29150238712463061, 'max_delta_step': 3, 'max_depth': 7, 'min_child_weight': 8, 'n_estimators': 44, 'reg_alpha': 6.6438489428781935, 'reg_lambda':

In [ ]:
# Model 2 (logloss)
xgbparams = {'num_class': 10}
xgb_model =  xgb.XGBClassifier(
                         objective = 'multi:softprob',
                         learning_rate= 0.1, 
                         n_estimators = 50,
                         gamma=0.1, 
                         max_depth = 12,
                         min_child_weight = 20,
                         max_delta_step= 50,
                         subsample = 0.9,
                         colsample_bytree = 0.7,
                         reg_lambda = 0.1,
                         reg_alpha = 0.1,
                         random_state=12345,
                         **xgbparams)
xgb_model = xgb_model.fit(train_data, train_labels, eval_metric = 'mlogloss')
results = xgb_model.predict_proba(test_data)
for i in range(10):
    submission_log.iloc[:,(i+1)] = results[:,i]
submission_log.to_csv('xgb_solution_logloss_1.csv', index= False)

Score: 0.17534